# From cat to dog

#### *A simple and light-hearted motivating example for algorithmic recourse*

In [27]:
using Pkg; Pkg.activate("../..")

  Activating environment at `~/Documents/code/AlgorithmicRecourse.jl/docs/Project.toml`


In [28]:
using CounterfactualExplanations, Flux
data = CounterfactualExplanations.Data.cats_dogs()
X, y = values(data)
cats = findall(y.==0)
dogs = findall(y.==1)
x = Flux.unstack(X,2)
data = zip(x,y);

In [29]:
using Images, FileIO, GLMakie
cat = load("www/cat.png")
dog = load("www/dog.png")
ms = 50
marker = map(y -> y==0 ? cat : dog,y);

In [30]:
using GLMakie
plt = scatter(X[1,cats],X[2,cats],marker=cat,markersize=ms)
scatter!(X[1,dogs],X[2,dogs],marker=dog,markersize=ms)
save("www/samples.png", plt);

![](www/samples.png)

## The black-box systems

### MLP

In [31]:
nn = Chain(Dense(2,1))
λ = 1
sqnorm(x) = sum(abs2, x)
weight_regularization(λ=λ) = 1/2 * λ^2 * sum(sqnorm, Flux.params(nn))
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) + weight_regularization();

In [32]:
using Flux.Optimise: update!, ADAM
using Statistics
opt = ADAM()
epochs = 10

for epoch = 1:epochs
  for d in data
    gs = gradient(params(nn)) do
      l = loss(d...)
    end
    update!(opt, params(nn), gs)
  end
end

In [33]:
using CounterfactualExplanations, CounterfactualExplanations.Models
import CounterfactualExplanations.Models: logits, probs # import functions in order to extend

# Step 1)
struct NeuralNetwork <: Models.FittedModel
    nn::Any
end

# Step 2)
logits(𝑴::NeuralNetwork, X::AbstractArray) = 𝑴.nn(X)
probs(𝑴::NeuralNetwork, X::AbstractArray)= σ.(logits(𝑴, X))
𝑴 = NeuralNetwork(nn);

### Laplace approximation

In [34]:
# using BayesLaplace
# la = laplace(nn, λ=λ)
# fit!(la, data);

In [35]:
# # Step 1)
# struct LaplaceNeuralNetwork <: Models.FittedModel
#     la::BayesLaplace.LaplaceRedux
# end

# # Step 2)
# logits(𝑴::LaplaceNeuralNetwork, X::AbstractArray) = 𝑴.la.model(X)
# probs(𝑴::LaplaceNeuralNetwork, X::AbstractArray)= BayesLaplace.predict(𝑴.la, X)
# 𝑴ᴸ = LaplaceNeuralNetwork(la);

In [36]:
using GLMakie
function plot_contour_makie(X,y,𝑴;clegend=true,title="",length_out=50,zoom=-1,xlim=nothing,ylim=nothing,linewidth=0.1)
    
    # Surface range:
    if isnothing(xlim)
        xlim = (minimum(X[:,1]),maximum(X[:,1])).+(zoom,-zoom)
    else
        xlim = xlim .+ (zoom,-zoom)
    end
    if isnothing(ylim)
        ylim = (minimum(X[:,2]),maximum(X[:,2])).+(zoom,-zoom)
    else
        ylim = ylim .+ (zoom,-zoom)
    end
    x_range = collect(range(xlim[1],stop=xlim[2],length=length_out))
    y_range = collect(range(ylim[1],stop=ylim[2],length=length_out))
    Z = [Models.probs(𝑴,[x, y])[1] for x=x_range, y=y_range]

    # Plot:
    plt = contourf(
        x_range, y_range, Z; 
        colorbar=clegend, title=title, linewidth=linewidth,
        xlim=xlim,
        ylim=ylim
    )
    
    scatter!(X'[1,cats],X'[2,cats],marker=cat,markersize=ms)
    scatter!(X'[1,dogs],X'[2,dogs],marker=dog,markersize=ms)

    return plt
end

plot_contour_makie (generic function with 1 method)

In [37]:
plt = plot_contour_makie(X',y,𝑴,title="MLP")
save("www/predictive_mlp.png", plt)
# plt = plot_contour_makie(X',y,𝑴ᴸ,title="Laplace")
# save("www/predictive_laplace.png", plt)

GLMakie.Screen(...)

### Generating recourse

In [38]:
using Random
Random.seed!(1234)
cats = findall(y.==0)
x̅ = X[:,rand(cats)]
y̅ = 0.0
target = 1.0 # opposite label as target
γ = 0.75; # desired level of confidence

In [39]:
generator = GenericGenerator(0.01,2,1e-5,:logitbinarycrossentropy,nothing)
recourse = generate_counterfactual(generator, x̅, 𝑴, target, γ); # generate recourse

In [40]:
T = size(recourse.path)[1]
X_path = reduce(hcat,recourse.path)
ŷ = CounterfactualExplanations.target_probs(probs(recourse.𝑴, X_path),target)
plt = plot_contour_makie(X',y,𝑴)
for t in 1:T
    scatter!([recourse.path[t][1]], [recourse.path[t][2]],marker=cat,markersize=ms)    
end
save("www/recourse_mlp.png",plt)

GLMakie.Screen(...)

In [41]:
T = size(recourse.path)[1]
X_path = reduce(hcat,recourse.path)
ŷ = CounterfactualExplanations.target_probs(probs(recourse.𝑴, X_path),target)
fig, ax, contourplot = plot_contour_makie(X',y,𝑴)

# animation settings
timestamps = range(1, T, step=1)

record(fig, "www/recourse_mlp.gif", timestamps; framerate = 30) do t
    scatter!([recourse.path[Int(t)][1]], [recourse.path[Int(t)][2]],marker=cat,markersize=ms)
end

"www/recourse_mlp.gif"

In [42]:
# generator = GreedyGenerator(1,30,:logitbinarycrossentropy,nothing)
# recourse = generate_counterfactual(generator, x̅, 𝑴ᴸ, target, γ); # generate recourse

In [43]:
# T = size(recourse.path)[1]
# X_path = reduce(hcat,recourse.path)
# ŷ = CounterfactualExplanations.target_probs(probs(recourse.𝑴, X_path),target)
# plt = plot_contour_makie(X',y,𝑴ᴸ)
# for t in 1:T
#     scatter!([recourse.path[t][1]], [recourse.path[t][2]],marker=cat,markersize=ms)    
# end
# save("www/recourse_laplace.png",plt)

GLMakie.Screen(...)

In [45]:
# T = size(recourse.path)[1]
# X_path = reduce(hcat,recourse.path)
# ŷ = CounterfactualExplanations.target_probs(probs(recourse.𝑴, X_path),target)
# fig, ax, contourplot = plot_contour_makie(X',y,𝑴ᴸ)

# # animation settings
# timestamps = range(1, T, step=1)

# record(fig, "www/recourse_laplace.gif", timestamps; framerate = 15) do t
#     scatter!([recourse.path[Int(t)][1]], [recourse.path[Int(t)][2]],marker=cat,markersize=ms)
# end

"www/recourse_laplace.gif"